In [ ]:
%%bash
nrnivmodl

In [ ]:
# Download NEURON: http://www.neuron.yale.edu/neuron/download
# Download PyNeuronToolbox: https://github.com/ahwillia/PyNeuron-Toolbox

from neuron import h
import numpy as np
import math
import pylab as plt
import scipy.linalg
from PyNeuronToolbox.record import ez_record,ez_convert
#from PyNeuronToolbox import neuromorpho,morphology
from PyNeuronToolbox.morphology import shapeplot,shapeplot_animate,load
from mpl_toolkits.mplot3d import Axes3D
from JSAnimation import IPython_display
from matplotlib import animation
from matplotlib.pyplot import cm
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning)
%matplotlib inline
np.random.seed(123456789)

import re
import pandas as pd

DC = 10 # diffusion coeff [microns^2 s^-1]

cellname = 'dgyrus'
#cellname = 'granule'
#neuromorpho.download('041015-vehicle1', filename=cellname+'_morph.swc')
#cell = morphology.load(cellname+'_morph.swc',use_axon=False)
#cell = load(cellname+'_morph.swc',use_axon=False)


In [ ]:
import sys
sys.path.append('../')
sys.path.append('../simulation/')
import sushibelt
#import DG_10reg_10dv_model as model
import DG_10reg_1dv_model as model
import time
tnorm = np.sum(model.target ** 2)

In [ ]:
N=model.N
Ndim = len(model.lowb)
(N,max(5 * Ndim, 15),151*max(5 * Ndim, 15))

In [ ]:
tdf=pd.read_csv('../data/DG_seg_mapping.csv')
abbDG=tdf['abb']
abbT={}
segIdx={}
for i in range(N):
    abbT[abbDG[i]] = 1+ abbT.get(abbDG[i],0)
    ll=segIdx.get(abbDG[i],[])
    ll.append(i)
    segIdx[abbDG[i]] = ll


In [ ]:
from PyNeuronToolbox.morphology import dist_between,allsec_preorder

def dist_to_soma(segment):
    return dist_between(h,allsec_preorder(h)[0](0.5),segment)

#seglist in pre-order
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    locs = np.linspace(0,1,sec.nseg+2)[1:-1]
    for loc in locs:
        seg_list.append(sec(loc))
n = len(seg_list)
N = n

In [ ]:
soma = allsec_preorder(h)[0]
soma(0.5)

In [ ]:
(N,seg_list)

In [ ]:
for i in range(len(sec_list)):
    sec = sec_list[i]
    print(f"{i}: {sec.name()}, nseg={sec.nseg}, propNseg={np.round(sec.L/3.7)}, L={sec.L}, or={sec.orientation()}, hname={sec.hname()}")

In [ ]:
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    if sec.name() != 'dgyrus_morph.soma[0]':
        nseg = int(np.round(sec.L/3.7))
        locs = np.linspace(0,1,nseg+2)[1:-1]
        for loc in locs:
            seg_list.append(sec(loc))
    else:
        locs = np.linspace(0,1,3)[1:-1]
        for loc in locs:
            seg_list.append(sec(loc))
n = len(seg_list)
N = n
N

In [ ]:
(dist_to_soma(seg_list[0]),dist_to_soma(seg_list[1]),sec.L,np.linspace(0,1,sec.nseg+2),np.linspace(0,1,sec.nseg+2)[1:-1])

In [ ]:
dts = [dist_to_soma(s) for s in seg_list]
max_dts = np.max(dts) + 0.01
dts_bounds = np.linspace(0,max_dts,11)
(dts_bounds,dts_bounds[0],dts_bounds[1])

In [ ]:
p = re.compile("^([^\\[]+)\\[.+")
tps = [p.sub("\\1",seg.sec.name()) for seg in seg_list]
pd.Series(tps).drop_duplicates().tolist()

In [ ]:
seg_list = []
seg_abb = []
seg_name = []
seg_dts = [dist_to_soma(s) for s in seg_list]
for sec in sec_list:
    if sec.name() != 'dgyrus_morph.soma[0]':
        nseg = int(np.round(sec.L/3.7))
        locs = np.linspace(0,1,nseg+2)[1:-1]
        for loc in locs:
            seg_list.append(sec(loc))
            seg_name.append(sec.name())
            ldts = dist_to_soma(sec(loc))
            seg_dts.append(ldts)
            for j in range(len(dts_bounds)-1):
                print(f"ldts={ldts},j={j},dts_bounds[j]={dts_bounds[j]} and dts_bounds[j+1]={dts_bounds[j+1]}")
                if ldts >= dts_bounds[j] and ldts < dts_bounds[j+1]:
                    print(f"\trd{j+1},len(seg_abb)={len(seg_abb)}")
                    seg_abb.append(f"rd{j+1}")
                    break                    
    else:
        locs = np.linspace(0,1,3)[1:-1]
        for loc in locs:
            seg_list.append(sec(loc))
            seg_name.append(sec.name())
            ldts = dist_to_soma(sec(loc))
            seg_dts.append(ldts)
            for j in range(len(dts_bounds)-1):
                print(f"ldts={ldts},j={j},dts_bounds[j]={dts_bounds[j]} and dts_bounds[j+1]={dts_bounds[j+1]}")
                if ldts >= dts_bounds[j] and ldts < dts_bounds[j+1]:
                    print(f"\trd{j+1},len(seg_abb)={len(seg_abb)}")
                    seg_abb.append(f"rd{j+1}")
                    break
n = len(seg_list)
N = n
print([len(seg_abb),len(seg_name),len(seg_dts),len([i for i in range(N)])])

In [ ]:
(len(seg_name),len(seg_abb),len(seg_dts),len(sec_list),seg_abb)

In [ ]:
abbDG=tdf['abb']
abbT={}
segIdx={}
for i in range(N):
    abbT[abbDG[i]] = 1+ abbT.get(abbDG[i],0)
    ll=segIdx.get(abbDG[i],[])
    ll.append(i)
    segIdx[abbDG[i]] = ll


In [ ]:
#expD=pd.read_csv('../data/DG_intensity_values_15072024.csv')
expD=pd.read_csv('../data/DG_density_values_15072024.csv')
bgSignal=1e-5
#cname='D0W3'
cname0='D0M3'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

#cname='D7W3'
cname7='D7M3'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #results to fit to
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point


In [ ]:
parnames=['F','Ctau','mProp','demand_rd1','demand_rd2','demand_rd3','demand_rd4','demand_rd5','demand_rd6',
          'demand_rd7','demand_rd8','demand_rd9','demand_rd10','dv_rd1','dv_rd2','dv_rd3','dv_rd4','dv_rd5',
          'dv_rd6','dv_rd7','dv_rd8','dv_rd9','dv_rd10']
lowb=np.array([0,-18,0,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,
               1.0e-07,-18,-18,-18,-18,-18,-18,-18,-18,-18,-18,])
upbga=np.array([1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
(len(parnames),len(lowb),len(upbga),parnames[3],parnames[13])

In [ ]:
len(sec_list)

In [ ]:
(model.lowb[13],model.upbga[13])

In [ ]:
#par =  [0.0004948517630019271,-2.901410577587942,0.49806692055122903,0.3209152647410538,0.41607574160259897,0.13475965672949675,0.19456334897464378,0.9995097181530779,0.0013403410023172134,0.00023786383238157604,0.00035606148539841663,0.00040396058440435756,0.000545230888035824,-5.320029850842932]
#cost = 3.546758073855645e-05
#chi2 = 42.40872728534614
par =  [0.010254232013526715,-5.638032179385583,0.4901989246102767,1.0346217672476097e-07,0.029242261750488406,0.01218622680694604,0.041390587450443085,0.06704294550203627,0.08976537459084287,0.1513671679744187,0.6105826545927185,0.543743857008341,0.28125287183335723,-5.336672864778542]
cost = 1.5127512266643512e-07
chi2 = 0.2911841351618986
mProp = par[2]
utrace = model.calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], model.segIdx, model.expD['Abbreviation'], fun=np.sum)


In [ ]:
len(par)

In [ ]:
{'F' : par[0],
'Ctau' : par[1],
'mProp' : par[2],
'demand_rd1' : par[3],
'demand_rd2' : par[4],
'demand_rd3' : par[5],
'demand_rd4' : par[6],
'demand_rd5' : par[7],
'demand_rd6' : par[8],
'demand_rd7' : par[9],
'demand_rd8' : par[10],
'demand_rd9' : par[11],
'demand_rd10' : par[12],
'dv' : 10 ** par[13],
'T1/2': np.log(2)*7/((10 ** par[13]) * day7),
#'dv_rd1' : par[13],
#'dv_rd2' : par[14],
#'dv_rd3' : par[15],
#'dv_rd4' : par[16],
#'dv_rd5' : par[17],
#'dv_rd6' : par[18],
#'dv_rd7' : par[19],
#'dv_rd8' : par[20],
#'dv_rd9' : par[21],
#'dv_rd10' : par[22],
'norm_factor' : (1-mProp),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - model.target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - model.target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - model.target)/model.targSD) ** 2),
'cost' : cost,
'Chi2' : chi2
}

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,label='Target')
ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
#ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_soma.png')
#plt.savefig('bestFitCA1_20reg_1dv_soma.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_soma.svg')

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label='Target')
#ax.plot(x,resM,label='mobile DC')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFit_DG_10reg_1dv_density.png')
#plt.savefig('bestFit_DG_10reg_1dv_density.pdf')
#plt.savefig('bestFit_DG_10reg_1dv_density.svg')

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,expD[f"{cname0}_MEAN"],label=cname0)
#ax.plot(x,expD[f"{cname7}_MEAN"],label=cname7)
#ax.plot(x,resM,label='mobile DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_soma.png')
#plt.savefig('bestFitCA1_20reg_1dv_soma.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_soma.svg')

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
#ax.plot(x,expD[f"{cname0}_MEAN"],label=cname0)
ax.plot(x,expD[f"{cname7}_MEAN"],label=cname7)
#ax.plot(x,resM,label='mobile DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_soma.png')
#plt.savefig('bestFitCA1_20reg_1dv_soma.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_soma.svg')

In [ ]:
fig, ax = plt.subplots()
#plt.plot(x,target,label=cname7)
ax.errorbar(x,target,targSD,label=cname7, fmt='o')
#plt.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFit_DG_10reg_1dv_density_target_error.png')
#plt.savefig('bestFit_DG_10reg_1dv_density_target_error.pdf')
#plt.savefig('bestFit_DG_10reg_1dv_density_target_error.svg')

# Three weeks

In [ ]:
expD=pd.read_csv('../data/DG_density_values_15072024.csv')
bgSignal=1e-5

cname0='D0W3'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

cname7='D7W3'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #results to fit to
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point


In [ ]:
(expD[f"{cname7}_MEAN"],target,model.target)

In [ ]:
model.dinit=dinit
utrace = model.calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], model.segIdx, model.expD['Abbreviation'], fun=np.sum)

In [ ]:
{'F' : par[0],
'Ctau' : par[1],
'mProp' : par[2],
'demand_rd1' : par[3],
'demand_rd2' : par[4],
'demand_rd3' : par[5],
'demand_rd4' : par[6],
'demand_rd5' : par[7],
'demand_rd6' : par[8],
'demand_rd7' : par[9],
'demand_rd8' : par[10],
'demand_rd9' : par[11],
'demand_rd10' : par[12],
'dv' : 10 ** par[13],
'T1/2': np.log(2)*7/((10 ** par[13]) * day7),
#'dv_rd1' : par[13],
#'dv_rd2' : par[14],
#'dv_rd3' : par[15],
#'dv_rd4' : par[16],
#'dv_rd5' : par[17],
#'dv_rd6' : par[18],
#'dv_rd7' : par[19],
#'dv_rd8' : par[20],
#'dv_rd9' : par[21],
#'dv_rd10' : par[22],
'norm_factor' : (1-mProp),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost,
'Chi2' : chi2
}

In [ ]:
[par[13],10 ** par[13], np.log(2)*7/(10 ** par[13] * day7)]

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label='Target')
#ax.plot(x,resM,label='mobile DC')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFit_DG_10reg_1dv_density_3W_target.png')
#plt.savefig('bestFit_DG_10reg_1dv_density_3W_target.pdf')
#plt.savefig('bestFit_DG_10reg_1dv_density_3W_target.svg')

In [ ]:
fig, ax = plt.subplots()
#plt.plot(x,target,label=cname7)
ax.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFit_DG_10reg_1dv_density_3W_target_error.png')
#plt.savefig('bestFit_DG_10reg_1dv_density_3W_target_error.pdf')
#plt.savefig('bestFit_DG_10reg_1dv_density_3W_target_error.svg')

# 18 months

In [ ]:
#expD=pd.read_csv('../data/DG_intensity_values_15072024.csv')
expD=pd.read_csv('../data/DG_density_values_15072024.csv')
bgSignal=1e-5
cname0='D0M18'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

cname7='D7M18'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #results to fit to
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point


In [ ]:
model.dinit=dinit
utrace = model.calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], model.segIdx, model.expD['Abbreviation'], fun=np.sum)

In [ ]:
{'F' : par[0],
'Ctau' : par[1],
'mProp' : par[2],
'demand_rd1' : par[3],
'demand_rd2' : par[4],
'demand_rd3' : par[5],
'demand_rd4' : par[6],
'demand_rd5' : par[7],
'demand_rd6' : par[8],
'demand_rd7' : par[9],
'demand_rd8' : par[10],
'demand_rd9' : par[11],
'demand_rd10' : par[12],
'dv' : 10 ** par[13],
'T1/2': np.log(2)*7/((10 ** par[13]) * day7),
#'dv_rd1' : par[13],
#'dv_rd2' : par[14],
#'dv_rd3' : par[15],
#'dv_rd4' : par[16],
#'dv_rd5' : par[17],
#'dv_rd6' : par[18],
#'dv_rd7' : par[19],
#'dv_rd8' : par[20],
#'dv_rd9' : par[21],
#'dv_rd10' : par[22],
'norm_factor' : (1-mProp),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost,
'Chi2' : chi2
}

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label='Target')
#ax.plot(x,resM,label='mobile DC')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFit_DG_10reg_1dv_density_18M_target.png')
#plt.savefig('bestFit_DG_10reg_1dv_density_18M_target.pdf')
#plt.savefig('bestFit_DG_10reg_1dv_density_18M_target.svg')

In [ ]:
fig, ax = plt.subplots()
#plt.plot(x,target,label=cname7)
ax.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFit_DG_10reg_1dv_density_18M_target_error.png')
#plt.savefig('bestFit_DG_10reg_1dv_density_18M_target_error.pdf')
#plt.savefig('bestFit_DG_10reg_1dv_density_18M_target_error.svg')